In [1]:
import pandas as pd

# Replace 'your_file.csv' with the actual path to your CSV file
df = pd.read_csv('/content/cleaned_dataset.csv')

# Display the columns
print("Columns:")
print(df.columns)

# Display the datatypes of each column
print("\nDatatypes:")
print(df.dtypes)

# Display the first 5 rows
print("\nFirst 5 rows:")
print(df.head())

# Display the last 5 rows
print("\nLast 5 rows:")
print(df.tail())

Columns:
Index(['time', 'btc__current_supply_native_units', 'btc__market_cap_usd',
       'btc__usd_4_00pm_new_york_price__index_value',
       'btc__usd_11_00am_london_price__index_value',
       'eth__current_supply_native_units', 'eth__market_cap_usd',
       'eth__usd_11_00am_london_price__index_value',
       'eth__usd_4_00pm_new_york_price__index_value',
       'usdc__usd_4_00pm_new_york_price__index_value',
       'usdc__usd_11_00am_london_price__index_value',
       'usdc__current_supply_native_units', 'usdc__market_cap_usd',
       'usdt__usd_4_00pm_new_york_price__index_value',
       'usdt__usd_11_00am_london_price__index_value',
       'xrp__current_supply_native_units',
       'xrp__usd_11_00am_london_price__index_value',
       'xrp__usd_4_00pm_new_york_price__index_value', 'xrp__market_cap_usd',
       'ada__current_supply_native_units',
       'ada__usd_11_00am_london_price__index_value',
       'ada__usd_4_00pm_new_york_price__index_value', 'ada__market_cap_usd',
     

In [2]:
import os
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt

# Load the dataset
file_path = "/content/cleaned_dataset.csv"  # Replace with the path to your dataset
data = pd.read_csv(file_path)

# Convert the 'time' column to datetime format and filter data from 2019 onwards
data['time'] = pd.to_datetime(data['time'], format='%d-%m-%Y')
data = data[data['time'] >= '2019-01-01']
data.sort_values('time', inplace=True)

# Directory to save all outputs
output_folder = "prophet_forecasts"
os.makedirs(output_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Initialize a DataFrame to store historical and future predictions
results = pd.DataFrame()

# Define the period for the historical values (last year)
historical_start_date = data['time'].max() - pd.DateOffset(years=4)

# Iterate through each numeric column for Prophet modeling
numeric_columns = data.select_dtypes(include=[np.number]).columns

for column in numeric_columns:
    print(f"Processing column: {column}")

    # Prepare data for Prophet
    prophet_data = data[['time', column]].dropna()
    prophet_data.columns = ['ds', 'y']  # Rename columns to match Prophet's expected format

    try:
        # Initialize and fit the Prophet model with daily seasonality enabled
        model = Prophet(daily_seasonality=True, yearly_seasonality=True)
        model.fit(prophet_data)

        # Create a DataFrame for future predictions (next year only)
        future = model.make_future_dataframe(periods=365, freq='D')
        forecast = model.predict(future)

        # Filter historical data (last year)
        historical_data = prophet_data[(prophet_data['ds'] >= historical_start_date)]
        historical_data['type'] = 'historical'
        historical_data.rename(columns={'y': f"{column}_value"}, inplace=True)

        # Filter future predictions
        future_forecast = forecast[forecast['ds'] > prophet_data['ds'].max()]
        future_forecast = future_forecast[['ds', 'yhat']].rename(
            columns={'yhat': f"{column}_value"}
        )
        future_forecast['type'] = 'predicted'

        # Combine historical and future data
        combined = pd.concat([historical_data[['ds', f"{column}_value", 'type']],
                              future_forecast], ignore_index=True)

        # Add the column-specific data to the results DataFrame
        if results.empty:
            results['time'] = combined['ds']
        results[f"{column}_value"] = combined[f"{column}_value"]
        results[f"{column}_type"] = combined['type']

        # Plot the forecast
        plt.figure(figsize=(12, 6))
        plt.plot(prophet_data['ds'], prophet_data['y'], label="Historical Data", color="blue")
        plt.plot(forecast['ds'], forecast['yhat'], label="Predicted Data", color="orange", linestyle="--")
        plt.fill_between(
            forecast['ds'],
            forecast['yhat_lower'],
            forecast['yhat_upper'],
            color="gray",
            alpha=0.2,
            label="Confidence Interval"
        )
        plt.title(f"Prophet Forecast for {column} (Last Year & Next Year)")
        plt.xlabel("Time")
        plt.ylabel(column)
        plt.legend()
        plt.grid()

        # Save the plot to the output folder
        plot_path = os.path.join(output_folder, f"{column}_prophet_forecast.png")
        plt.savefig(plot_path)
        plt.close()
    except Exception as e:
        print(f"Error processing column {column}: {e}")

# Save the combined results to a CSV file in the output folder
csv_path = os.path.join(output_folder, 'prophet_historical_and_next_year.csv')
results.to_csv(csv_path, index=False)
print(f"Combined historical and next year predictions saved to '{csv_path}'.")
print(f"All forecast plots saved in the folder: '{output_folder}'.")


Processing column: btc__current_supply_native_units


DEBUG:cmdstanpy:input tempfile: /tmp/tmpumruwf8i/1ug0uxtd.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpumruwf8i/fqn8by8m.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=29514', 'data', 'file=/tmp/tmpumruwf8i/1ug0uxtd.json', 'init=/tmp/tmpumruwf8i/fqn8by8m.json', 'output', 'file=/tmp/tmpumruwf8i/prophet_model4z1q59ci/prophet_model-20241124191406.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:14:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:14:13 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
<ipython-input-2-8e091560cd07>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

Processing column: btc__market_cap_usd


DEBUG:cmdstanpy:input tempfile: /tmp/tmpumruwf8i/j3o9puy5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=25089', 'data', 'file=/tmp/tmpumruwf8i/_hgz1km5.json', 'init=/tmp/tmpumruwf8i/j3o9puy5.json', 'output', 'file=/tmp/tmpumruwf8i/prophet_modelbxor0_9c/prophet_model-20241124191414.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:14:14 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:14:15 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
<ipython-input-2-8e091560cd07>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

Processing column: btc__usd_4_00pm_new_york_price__index_value


...

Processing column: trx__usd_4_00pm_new_york_price__index_value


DEBUG:cmdstanpy:input tempfile: /tmp/tmpumruwf8i/_4ss__22.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=19930', 'data', 'file=/tmp/tmpumruwf8i/oms45y0x.json', 'init=/tmp/tmpumruwf8i/_4ss__22.json', 'output', 'file=/tmp/tmpumruwf8i/prophet_modelunr_mw_n/prophet_model-20241124191522.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:15:22 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:15:23 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
<ipython-input-2-8e091560cd07>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

Processing column: trx__usd_11_00am_london_price__index_value


DEBUG:cmdstanpy:input tempfile: /tmp/tmpumruwf8i/1zzp2shh.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=65361', 'data', 'file=/tmp/tmpumruwf8i/yi_dsqvy.json', 'init=/tmp/tmpumruwf8i/1zzp2shh.json', 'output', 'file=/tmp/tmpumruwf8i/prophet_modelcl6j7sot/prophet_model-20241124191524.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:15:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:15:25 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
<ipython-input-2-8e091560cd07>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

Combined historical and next year predictions saved to 'prophet_forecasts/prophet_historical_and_next_year.csv'.
All forecast plots saved in the folder: 'prophet_forecasts'.


In [ ]:
!zip -r /content/prophet_forecasts.zip /content/prophet_forecasts